In [1]:
# 1. Imports and Global Definitions

import dynamiqs as dq
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Define units (frequencies in GHz)
GHz = 1.0
MHz = 1e-3 * GHz
kHz = 1e-3 * MHz

In [2]:
# 2. System Parameters (Circuit and Dissipation)

# Bare resonant frequencies
omega_a_0 = 5.26 * GHz  # mode a (memory)
omega_b_0 = 7.7 * GHz   # mode b (buffer)

# Zero-point fluctuation phases
phi_a = 0.06  # mode a
phi_b = 0.29  # mode b

# Josephson energies
e_J = 12.03 * GHz
delta_e_J = 0.47 * GHz

# Dissipation rates (loss)
kappa_a = 9.3 * kHz
kappa_b = 2.6 * MHz

# Target parameters
g2_target = 0.763 * MHz  # target two-photon exchange rate
alpha_target = np.sqrt(5.0)  # target amplitude (or equivalently, number of photons)

# Driving strengths (nominal values)
epsilon_p = 2 * g2_target / (e_J * phi_a**2 * phi_b)  # flux pump power
epsilon_d = -alpha_target**2 * np.conj(g2_target)      # buffer pump power


In [3]:
# 3. AC Stark Shift and Frequency Recalibration

# Initialize Stark shifts and matching condition
delta_a, delta_b = 0.0, 0.0
omega_p = 2 * omega_a_0 - omega_b_0  # initial frequency matching

# Recursively compute the AC Stark shifts over rec_depth iterations
rec_depth = 30
for i in range(rec_depth):
    alpha_1 = 1j * e_J * epsilon_p * phi_a / (kappa_a/2 + 1j * (omega_a_0 - omega_p))
    alpha_2 = 1j * e_J * epsilon_p * phi_a / (kappa_a/2 + 1j * (omega_a_0 + omega_p))
    beta_1  = 1j * e_J * epsilon_p * phi_b / (kappa_b/2 + 1j * (omega_b_0 - omega_p))
    beta_2  = 1j * e_J * epsilon_p * phi_b / (kappa_b/2 + 1j * (omega_b_0 + omega_p))

    pre = (np.real(alpha_1) + np.real(alpha_2)) * phi_a + \
          (np.real(beta_1) + np.real(beta_2)) * phi_b

    delta_a = (1/3) * e_J * epsilon_p * phi_a**2 * pre
    delta_b = (1/3) * e_J * epsilon_p * phi_b**2 * pre

    omega_a = omega_a_0 - 2 * delta_e_J * phi_a**2 + delta_a
    omega_b = omega_b_0 - 2 * delta_e_J * phi_b**2 + delta_b
    omega_p = 2 * omega_a - omega_b

# Set buffer drive frequency
omega_d = omega_b

print("Updated omega_a =", omega_a)
print("Updated omega_b =", omega_b)
print("Updated omega_p =", omega_p)
print("Buffer drive frequency omega_d =", omega_d)


Updated omega_a = 5.2566862215107335
Updated omega_b = 7.622586452514629
Updated omega_p = 2.8907859905068376
Buffer drive frequency omega_d = 7.622586452514629


In [4]:
# 4. Hilbert Space Operators and Identity

# Fock-space dimensions
n_fock_a = 30  # mode a
n_fock_b = 7   # mode b

# Annihilation operators (tensor product representation)
a, b = dq.destroy(n_fock_a, n_fock_b)

# Identity operator on the full space
identity = dq.tensor(dq.eye(n_fock_a), dq.eye(n_fock_b))


In [5]:
# 5. Define the ATS Driving Term and Displaced–Rotated Operator

# The ATS driving term is nominally given by
eps_ats = lambda t: epsilon_p * jnp.cos(omega_p * t)

def op_rot_displaced(t, op, omega_rot, dis):
    """
    Constructs a time-dependent operator in the rotated and displaced frame.
    
    Parameters:
        t         : time
        op        : the bare operator (e.g., a or a.dag())
        omega_rot : rotation frequency for this mode
        dis       : displacement (complex number) for this mode
        
    Returns:
        The rotated–displaced operator.
    """
    return op * jnp.exp(-1j * omega_rot * t) + dis * identity

# For the rotating frame we choose:
omega_rot_a = omega_a_0  # mode a (memory) rotates at its (bare or updated) frequency
omega_rot_b = omega_d    # mode b (buffer) rotates at the drive frequency


In [6]:
# 6. Define the Full Circuit Hamiltonian (Rotated–Displaced Frame)


def hamiltonian_rotating_displaced(t):
    """
    Full circuit Hamiltonian in the rotated–displaced frame.
    
    This Hamiltonian contains five contributions:
        h_0           : bare mode energies
        h_ATS         : ATS drive term (with Taylor expansion approximations)
        h_buffer_drive: drive term on the buffer mode
        h_rot         : correction from the rotating frame derivative
        h_dis         : correction from the displaced frame derivative
    """
    # Compute displacement fields (from the AC Stark recalculation)
    alpha_t = alpha_1 * jnp.exp(-1j * omega_p * t) + alpha_2 * jnp.exp(1j * omega_p * t)
    beta_t  = beta_1  * jnp.exp(-1j * omega_p * t) + beta_2  * jnp.exp(1j * omega_p * t)
    
    # Time derivatives of displacement fields
    d_alpha_t = 1j * omega_p * (alpha_2 * jnp.exp(1j * omega_p * t) - alpha_1 * jnp.exp(-1j * omega_p * t))
    d_beta_t  = 1j * omega_p * (beta_2  * jnp.exp(1j * omega_p * t) - beta_1  * jnp.exp(-1j * omega_p * t))
    
    # Rotated–displaced operators for both modes
    a_f    = op_rot_displaced(t, a,    omega_rot_a, alpha_t)
    adag_f = op_rot_displaced(t, a.dag(), omega_rot_a, jnp.conj(alpha_t))
    b_f    = op_rot_displaced(t, b,    omega_rot_b, beta_t)
    bdag_f = op_rot_displaced(t, b.dag(), omega_rot_b, jnp.conj(beta_t))
    
    # Phase operator in the displaced–rotated frame
    phi_f = phi_a * (a_f + adag_f) + phi_b * (b_f + bdag_f)
    
    # Bare mode energies
    h_0 = omega_a_0 * (adag_f @ a_f) + omega_b_0 * (bdag_f @ b_f)
    
    # Corrections from the rotating frame derivative
    h_rot = -omega_rot_a * (dq.dag(a) @ a) - omega_rot_b * (dq.dag(b) @ b)
    
    # Corrections from the displaced frame derivative
    h_dis = (-1j * (d_alpha_t * dq.dag(a) * jnp.exp(1j * omega_rot_a * t)
                    - jnp.conj(d_alpha_t) * a * jnp.exp(-1j * omega_rot_a * t))
             -1j * (d_beta_t * dq.dag(b) * jnp.exp(1j * omega_rot_b * t)
                    - jnp.conj(d_beta_t) * b * jnp.exp(-1j * omega_rot_b * t)))
    
    # ATS term (using Taylor expansions)
    # Sine expansion: sin(ε) ≈ ε and sin(φ) ≈ φ - φ^3/6 gives:
    #   -2E_J ε φ + (1/3) E_J ε φ³.
    epsilon_val = eps_ats(t)
    h_sin = -2 * e_J * epsilon_val * phi_f + (1/3) * e_J * epsilon_val * (phi_f @ phi_f @ phi_f)
    
    # Cosine expansion: cos(ε) ≈ 1 - ε²/2 and cos(φ) ≈ I - φ²/2 gives:
    #   2ΔE_J [I - 0.5*(ε² I + φ²)].
    h_cos = 2 * delta_e_J * (identity - 0.5 * ((epsilon_val**2) * identity + (phi_f @ phi_f)))
    h_ats = h_sin + h_cos
    
    # Buffer drive term (non-rotating in this frame)
    h_buffer_drive = (jnp.conj(epsilon_d) * jnp.exp(1j * omega_d * t) * b_f +
                      epsilon_d * jnp.exp(-1j * omega_d * t) * bdag_f)
    
    return h_0 + h_rot + h_dis + h_ats + h_buffer_drive

# Make a time-callable JAX function for the lab–frame Hamiltonian.
H_lab = dq.timecallable(lambda t: hamiltonian_rotating_displaced(t))  # for reference


In [7]:
# 7. Define Collapse Operators and Initial State

# Collapse operators for both modes
c_ops = [jnp.sqrt(kappa_a) * a, jnp.sqrt(kappa_b) * b]

# Initial state: vacuum in both modes
psi0 = dq.tensor(dq.coherent(n_fock_a, 0.0), dq.coherent(n_fock_b, 0.0))


In [8]:
# 8. Simulation Time Grid

T = 50.0  # Total simulation time
tsave = jnp.linspace(0, T, 400)

In [10]:
# 9. Target Cat State Definition

def target_cat_state(alpha_val=alpha_target):
    """
    Constructs a target cat state in mode a (memory) and vacuum in mode b (buffer).

    The cat state is defined as a normalized superposition of two coherent states:
        |cat> = N ( |alpha> + |-alpha> ).

    Parameters:
        alpha_val : amplitude of the coherent states (default: alpha_target)

    Returns:
        A tensor product state: cat state in mode a ⊗ vacuum state in mode b.
    """
    # Create coherent states in mode a
    psi_alpha = dq.coherent(n_fock_a, alpha_val)
    psi_minus_alpha = dq.coherent(n_fock_a, -alpha_val)

    # Form the superposition of the two coherent states
    psi_cat = psi_alpha + psi_minus_alpha

    # Normalize the superposition.
    # Option 1: Use dq.norm if available.
    norm_val = dq.norm(psi_cat)
    psi_cat = psi_cat / norm_val

    # Option 2 (if dq.norm is not available):
    # norm_val = jnp.linalg.norm(jnp.asarray(psi_cat))
    # psi_cat = psi_cat / norm_val

    # Mode b is taken as the vacuum state.
    psi_b_vac = dq.coherent(n_fock_b, 0.0)
    
    return dq.tensor(psi_cat, psi_b_vac)

# Precompute the target cat state
psi_target = target_cat_state()


In [11]:
# 10. Optimal Control: Modify the Hamiltonian with Control Parameters

def hamiltonian_rd_opt(t, control):
    """
    Returns the full circuit Hamiltonian (rotated–displaced) with optimized control parameters.
    
    The control parameters are:
        control[0] = k_p, scaling factor for the ATS drive amplitude (flux pump)
        control[1] = k_d, scaling factor for the buffer drive amplitude
    
    These scaling factors multiply the nominal amplitudes (epsilon_p, epsilon_d) in the Hamiltonian.
    """
    k_p, k_d = control  # unpack control parameters

    # Compute displacement fields (remain as before)
    alpha_t = alpha_1 * jnp.exp(-1j * omega_p * t) + alpha_2 * jnp.exp(1j * omega_p * t)
    beta_t  = beta_1  * jnp.exp(-1j * omega_p * t) + beta_2  * jnp.exp(1j * omega_p * t)
    
    # Time derivatives of displacement fields
    d_alpha_t = 1j * omega_p * (alpha_2 * jnp.exp(1j * omega_p * t) - alpha_1 * jnp.exp(-1j * omega_p * t))
    d_beta_t  = 1j * omega_p * (beta_2  * jnp.exp(1j * omega_p * t) - beta_1  * jnp.exp(-1j * omega_p * t))
    
    # Rotated–displaced operators
    a_f    = op_rot_displaced(t, a,    omega_rot_a, alpha_t)
    adag_f = op_rot_displaced(t, a.dag(), omega_rot_a, jnp.conj(alpha_t))
    b_f    = op_rot_displaced(t, b,    omega_rot_b, beta_t)
    bdag_f = op_rot_displaced(t, b.dag(), omega_rot_b, jnp.conj(beta_t))
    
    # Displaced–rotated phase operator
    phi_f = phi_a * (a_f + adag_f) + phi_b * (b_f + bdag_f)
    
    # Bare mode energies
    h_0 = omega_a_0 * (adag_f @ a_f) + omega_b_0 * (bdag_f @ b_f)
    
    # Rotating frame correction
    h_rot = -omega_rot_a * (dq.dag(a) @ a) - omega_rot_b * (dq.dag(b) @ b)
    
    # Displaced frame derivative correction
    h_dis = (-1j * (d_alpha_t * dq.dag(a) * jnp.exp(1j * omega_rot_a * t)
                    - jnp.conj(d_alpha_t) * a * jnp.exp(-1j * omega_rot_a * t))
             -1j * (d_beta_t * dq.dag(b) * jnp.exp(1j * omega_rot_b * t)
                    - jnp.conj(d_beta_t) * b * jnp.exp(-1j * omega_rot_b * t)))
    
    # --- ATS Term with Optimized Control ---
    # Scale the nominal ATS drive amplitude (epsilon_p) by k_p.
    # The effective flux pump is: epsilon_eff(t) = k_p * epsilon_p * cos(omega_p*t)
    epsilon_eff = k_p * epsilon_p * jnp.cos(omega_p * t)
    
    # Sine expansion: -2E_J * epsilon_eff * phi_f + (1/3)E_J * epsilon_eff * phi_f³
    h_sin = -2 * e_J * epsilon_eff * phi_f + (1/3) * e_J * epsilon_eff * (phi_f @ phi_f @ phi_f)
    
    # Cosine expansion: 2ΔE_J [I - 0.5*(epsilon_eff² I + φ_f²)]
    h_cos = 2 * delta_e_J * (identity - 0.5 * ((epsilon_eff**2) * identity + (phi_f @ phi_f)))
    h_ats = h_sin + h_cos
    
    # --- Buffer Drive Term with Optimized Control ---
    # Scale the nominal buffer drive amplitude (epsilon_d) by k_d.
    effective_buffer = k_d * epsilon_d
    h_buffer_drive = (jnp.conj(effective_buffer) * jnp.exp(1j * omega_d * t) * b_f +
                      effective_buffer * jnp.exp(-1j * omega_d * t) * bdag_f)
    
    return h_0 + h_rot + h_dis + h_ats + h_buffer_drive

In [12]:
# 11. Simulation Wrapper for the Optimized Hamiltonian


def simulate_rd(control, tgrid=tsave):
    """
    Simulates the dynamics using the optimized rotated–displaced Hamiltonian.
    
    Parameters:
        control : array-like, [k_p, k_d] control scaling factors.
        tgrid   : time grid over which to simulate (default: tsave)
    
    Returns:
        res : the simulation result (states over time).
    """
    # Create a time-callable Hamiltonian with the given control parameters.
    H_rd_opt = dq.timecallable(lambda t: hamiltonian_rd_opt(t, control))
    
    # Run the simulation (mesolve) with the optimized Hamiltonian.
    res = dq.mesolve(H_rd_opt, c_ops, psi0, tgrid)
    return res


In [13]:
# 12. Objective Function for Optimal Control

def objective(control):
    """
    Objective function for optimal control.
    
    For a given set of control parameters (k_p and k_d), simulate the dynamics
    and compute the infidelity (1 - fidelity) between the final state and the target cat state.
    
    Parameters:
        control : array-like, control parameters [k_p, k_d].
        
    Returns:
        cost : scalar cost value (to be minimized).
    """
    res = simulate_rd(control)
    final_state = res.states[-1]
    fid = dq.fidelity(final_state, psi_target)
    cost = 1.0 - fid
    print(f"Control: {control}, Final Fidelity: {fid:.6f}, Cost: {cost:.6f}")
    return cost


In [ ]:
# 13. Run the Optimal Control Optimization

# Initial guess for control parameters: no scaling (i.e. 1.0 for both)
control_init = np.array([1.0, 1.0])

# Optimize using the Nelder-Mead algorithm (gradient-free; alternatives are possible)
result = minimize(objective, control_init, method='Nelder-Mead', options={'maxiter': 50, 'disp': True})

optimal_control = result.x
print("\nOptimal Control Parameters Found:")
print("k_p (ATS drive scaling) =", optimal_control[0])
print("k_d (Buffer drive scaling) =", optimal_control[1])

|██████████| 100.0% ◆ elapsed 06m20s ◆ remaining 0.00ms  


Control: [1. 1.], Final Fidelity: 0.144422, Cost: 0.855578


|██████████| 100.0% ◆ elapsed 07m10s ◆ remaining 0.00ms  


Control: [1.05 1.  ], Final Fidelity: 0.241027, Cost: 0.758973


|██████████| 100.0% ◆ elapsed 07m48s ◆ remaining 0.00ms  


Control: [1.   1.05], Final Fidelity: 0.213667, Cost: 0.786333


|█▋        |  17.3% ◆ elapsed 41.24s ◆ remaining 04m17s  

In [1]:
# 14. Simulate Dynamics with Optimal Control and Visualize Results

# Simulate with the optimal control parameters over the full time grid
res_opt = simulate_rd(optimal_control)

# Compute fidelity vs. time with the target cat state
fidelity_vs_time = []
for state in res_opt.states:
    fid = dq.fidelity(state, psi_target)
    fidelity_vs_time.append(fid)
fidelity_vs_time = np.array(fidelity_vs_time)

# Plot fidelity vs. time
plt.figure(figsize=(8, 6))
plt.plot(tsave, fidelity_vs_time, lw=2, label="Fidelity with Target Cat")
plt.xlabel("Time", fontsize=14)
plt.ylabel("Fidelity", fontsize=14)
plt.title("Evolution of Fidelity under Optimal Control", fontsize=16)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot the Wigner function (animation or snapshot) for mode a of the final state
# Here we plot a GIF of the Wigner functions over time.
dq.plot.wigner_gif(res_opt.states, fps=24)



NameError: name 'simulate_rd' is not defined